In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pydot
import os
import matplotlib.pyplot as plt
import copy
import time
from collections import Counter
import scene
import AllegroWrapper
import contact
import motion_planning
import search
import visualizations
from pydrake.all import StartMeshcat
from manipulation.utils import ConfigureParser

In [2]:
meshcat = None

In [4]:
#p_W_brick = [np.random.uniform(0.4, 0.75), np.random.uniform(-0.35, 0.35), 0.01]
#R_W_brick = None

p_W_brick = [2.75, 0.0, 0.01]
R_W_brick = [0, 0, 90]
meshcat, station, simulator, context, p_W_brick, R_W_brick = scene.init_scenario(brick_location=p_W_brick, brick_rotation=R_W_brick, meshcat=meshcat)

allegro = AllegroWrapper.AllegroHand(station, context)
simulator.set_target_realtime_rate(1.0)
x0 = station.GetOutputPort("iiwa+allegro.state_estimated").Eval(context)
# scene.render_station_diagram(station)

def reset_arm(station, simulator, context, x0):
    station.GetInputPort("iiwa+allegro.desired_state").FixValue(context, x0)
    simulator.AdvanceTo(context.get_time() + 1)


In [4]:
reset_arm(station, simulator, context, x0)

In [5]:
allegro.get_limbs()

dict_keys(['index_revolute_z', 'index_0', 'index_1', 'index_2', 'thumb_revolute_z', 'thumb_revolute_y', 'thumb_1', 'thumb_2', 'middle_revolute_z', 'middle_0', 'middle_1', 'middle_2', 'pinky_revolute_z', 'pinky_0', 'pinky_1', 'pinky_2'])

In [6]:
# allegro.set_positions(["index_1", "middle_1", "pinky_1"], [0.6] * 3)
# allegro.set_positions(["index_2", "middle_2", "pinky_2"], [0.3] * 3)

# simulator.AdvanceTo(context.get_time() + 1)

In [7]:
# allegro.get_positions(["thumb_revolute_y", "thumb_revolute_z"], context)

In [8]:
# allegro.set_positions(["thumb_revolute_z"], [-1])
# #allegro.set_positions(["thumb_0", "thumb_1"], [0.6, -2.6])
# simulator.AdvanceTo(context.get_time() + 1)


In [30]:
from pydrake.all import Solve, RotationMatrix, RigidTransform
from pydrake.multibody import inverse_kinematics

def optimize_arm_movement(X_WG, station, end_effector_poses, frame="hand_root"):
    """Convert end-effector pose list to joint position list using series of
    InverseKinematics problems. Note that q is 9-dimensional because the last 2 dimensions
    contain gripper joints, but these should not matter to the constraints.

    @param: X_WG (numpy array): X_WG[i] contains joint position q at index i.
    @param: station (pydrake.systems.framework.DiagramBuilder): DiagramBuilder containing the robot model.
    @param: end_effector_poses (python_list): end_effector_poses[i] contains the desired pose of the end-effector at index i.
    @param: frame (string): name of the frame of the end-effector.

    @return: q_knots (python_list): q_knots[i] contains IK solution that will give f(q_knots[i]) \approx pose_lst[i].
    """
    q_knots = []
    plant = station.GetSubsystemByName("plant")
    world_frame = plant.world_frame()
    gripper_frame = plant.GetFrameByName(frame)

    iiwa_initial = X_WG[:7]
    gripper_initial = np.ones((23))
    q_nominal = np.concatenate((iiwa_initial, gripper_initial))

    def AddPositionConstraint(ik, p_WG_lower, p_WG_upper):
        """Add position constraint to the ik problem. Implements an inequality
        constraint where f_p(q) must lie between p_WG_lower and p_WG_upper. Can be
        translated to
        ik.prog().AddBoundingBoxConstraint(f_p(q), p_WG_lower, p_WG_upper)
        """
        ik.AddPositionConstraint(
            frameA=world_frame,
            frameB=gripper_frame,
            p_BQ=np.zeros(3),
            p_AQ_lower=p_WG_lower,
            p_AQ_upper=p_WG_upper,
        )

    def AddOrientationConstraint(ik, R_WG, bounds):
        """Add orientation constraint to the ik problem. Implements an inequality
        constraint where the axis-angle difference between f_R(q) and R_WG must be
        within bounds. Can be translated to:
        ik.prog().AddBoundingBoxConstraint(angle_diff(f_R(q), R_WG), -bounds, bounds)
        """
        ik.AddOrientationConstraint(
            frameAbar=world_frame,
            R_AbarA=R_WG,
            frameBbar=gripper_frame,
            R_BbarB=RotationMatrix(),
            theta_bound=bounds,
        )

    for i in range(len(end_effector_poses)):
        ik = inverse_kinematics.InverseKinematics(plant)
        q_variables = ik.q()  # Get variables for MathematicalProgram
        prog = ik.prog()  # Get MathematicalProgram

        pose = end_effector_poses[i]
        AddPositionConstraint(
                    ik,
                    pose.translation() - 0.01 * np.ones(3),
                    pose.translation() + 0.01 * np.ones(3),
        )
        
        if frame == "hand_root":
            AddOrientationConstraint(ik, pose.rotation(), 0.01)
        
        
        prog.AddQuadraticErrorCost(np.identity(len(q_variables)), q_nominal, q_variables)
        if i == 0:
            prog.SetInitialGuess(q_variables, q_nominal)
        else:
            prog.SetInitialGuess(q_variables, q_knots[i-1])

        result_found = False
        for i in range(1000):
            result = Solve(prog)
            if result.is_success():
                result_found = True
                break

        if not result_found:
            raise RuntimeError("Inverse kinematics failed.")

        q_knots.append(result.GetSolution(q_variables))

    return np.array(q_knots)

def interpolate_locations(X_WG, p_WG_post, interp_steps=16, arc_height=0.1, rotate=False):
    p_WG_pre = X_WG.translation()
    xs = np.linspace(p_WG_pre[0], p_WG_post[0], interp_steps)
    ys = np.linspace(p_WG_pre[1], p_WG_post[1], interp_steps)
    zs = np.linspace(p_WG_pre[2], p_WG_post[2], interp_steps)
    interp_points = np.vstack((xs, ys, zs)).T

    poses = []
    for i in range(interp_steps):
        if rotate is None:
            rotation = X_WG.rotation()
        else:
            rotation = X_WG.rotation().MakeXRotation(np.pi / 2).MakeYRotation(np.pi / 2)
        p_WG = interp_points[i]
        poses.append(RigidTransform(rotation, p_WG))
    return poses

def move_arm(p_WG_post, simulator, station, context, poses=None, time_interval=0.5, frame="iiwa_link_6", arc_height=0.01):
    """
    Move the arm to a new location. If the arm is in contact with an object, stop moving.
    @param: X_WG (numpy array): Allegro wrapper with current robot state.
    @param: simulator (pydrake.systems.analysis.Simulator): Simulator object.
    @param: station (pydrake.systems.framework.DiagramBuilder): DiagramBuilder containing the robot model.
    @param: context (pydrake.systems.framework.Context): Context object.
    @param: end_effector_poses (python_list): end_effector_poses[i] contains the desired pose of the end-effector at index i.
    @param: time_interval (float): time interval for each step of the simulation.
    @param: frame (string): name of the frame of the end-effector.

    @return: The touched object and current contact params
    """    
    plant = station.GetSubsystemByName("plant")
    plant_context = plant.GetMyContextFromRoot(context)
    gripper = plant.GetBodyByName(frame)
    X_WG = plant.EvalBodyPoseInWorld(plant_context, gripper)
    if frame == "iiwa_link_6" and not poses:
        end_effector_poses = interpolate_locations(X_WG, p_WG_post, interp_steps=16, arc_height=arc_height)
    elif not poses:
        end_effector_poses = interpolate_locations(X_WG, p_WG_post, interp_steps=16, arc_height=arc_height, rotate=True)
    else:
        end_effector_poses = poses
        
    X_WG_full = station.GetOutputPort("iiwa+allegro.state_estimated").Eval(context)
    trajectory = optimize_arm_movement(X_WG_full, station, end_effector_poses, frame=frame)

    arm_trajectory = trajectory[:, :7]
    all_contacts = set()
    for state_update in arm_trajectory:
        new_state = X_WG_full.copy()
        new_state[:len(state_update)] = state_update
        station.GetInputPort("iiwa+allegro.desired_state").FixValue(context, new_state)
        simulator_steps = 25
        for i in range(simulator_steps):
            simulator.AdvanceTo(context.get_time() + time_interval / simulator_steps)
            # Check for contact
            current_contact = contact.get_contacts(station, context)
            for p_WC in current_contact:
                all_contacts.add(tuple(p_WC))
    return

In [127]:
def make_box_grip(box_location, box_rotation, allegro, station, context):
    # Do IK to get specific hand links (the end effectors) to pre-specified grasp points on the box
    # Grasp points are specified in the world frame

    # Get the box pose in the world frame
    p_W_box = box_location
    box_size = [0.08700, 0.0700, 0.026000]
    #box_size = [0.075, 0.05, 0.05]
    
    # Get frames of the hand links in the world frame
    plant = station.GetSubsystemByName("plant")
    plant_context = plant.GetMyContextFromRoot(context)
    world_frame = plant.world_frame()
    
    thumb_link = plant.GetBodyByName(allegro.link_ids["thumb_1"])
    index_link = plant.GetBodyByName(allegro.link_ids["index_2"])
    middle_link = plant.GetBodyByName(allegro.link_ids["middle_2"])
    pinky_link = plant.GetBodyByName(allegro.link_ids["pinky_2"])

    # Specify grasp points in the world frame
    p_W_thumb = p_W_box + np.array([-box_size[0] // 2, 0, 0])
    p_W_index = p_W_box + np.array([box_size[0] // 2, box_size[1] // 2, 0])
    p_W_middle = p_W_box + np.array([box_size[0] // 2, 0, 0])
    p_W_pinky = p_W_box + np.array([box_size[0] // 2, -box_size[1] // 2, 0])
    
    print(p_W_thumb, p_W_index, p_W_middle, p_W_pinky)
    # Generate IK constraints for each link
    ik = inverse_kinematics.InverseKinematics(plant)
    def add_ik_constraint(link, p_WQ):
        ik.AddPositionConstraint(
            frameA=world_frame,
            frameB=link.body_frame(),
            p_BQ=np.zeros(3),
            p_AQ_lower=p_WQ + np.array([-0.02, -0.02, -0.02]),
            p_AQ_upper=p_WQ + np.array([0.02, 0.02, 0.02])
        )

    add_ik_constraint(thumb_link, p_W_thumb)
    add_ik_constraint(index_link, p_W_index)
    add_ik_constraint(middle_link, p_W_middle)
    add_ik_constraint(pinky_link, p_W_pinky)    

    q_variables = ik.q()  # Get variables for MathematicalProgram
    prog = ik.prog()  # Get MathematicalProgram
    estimated_state = station.GetOutputPort("iiwa+allegro.state_estimated").Eval(context)
    q_nominal = np.concatenate([estimated_state[:7], np.ones(23)])
    prog.AddQuadraticErrorCost(np.identity(len(q_variables)), q_nominal, q_variables)

    result_found = False
    for i in range(1000):
        prog.SetInitialGuess(q_variables, q_nominal)
        result = Solve(prog)
        if result.is_success():
            result_found = True
            break

    if not result_found:
        raise RuntimeError("Inverse kinematics failed.")

    solution = result.GetSolution(q_variables)

    solution = np.concatenate([solution, np.zeros(16)])
    station.GetInputPort("iiwa+allegro.desired_state").FixValue(context, solution)
    simulator_steps = 100
    time_interval = 1.
    for i in range(simulator_steps):
        simulator.AdvanceTo(context.get_time() + time_interval / simulator_steps)

In [153]:
#p_W_brick = [np.random.uniform(0.4, 0.75), np.random.uniform(-0.35, 0.35), 0.01]
#R_W_brick = None

p_W_brick = [0.75, 0.0, 0.05]
R_W_brick = [90, 0, 90]
meshcat, station, simulator, context, p_W_brick, R_W_brick = scene.init_scenario(brick_location=p_W_brick, brick_rotation=R_W_brick, meshcat=meshcat)

allegro = AllegroWrapper.AllegroHand(station, context)
simulator.set_target_realtime_rate(1.0)
x0 = station.GetOutputPort("iiwa+allegro.state_estimated").Eval(context)
# scene.render_station_diagram(station)

def reset_arm(station, simulator, context, x0):
    station.GetInputPort("iiwa+allegro.desired_state").FixValue(context, x0)
    simulator.AdvanceTo(context.get_time() + 1)


In [15]:
reset_arm(station, simulator, context, x0)

In [71]:
####################
# Finger Walking
####################
reset_arm(station, simulator, context, x0)
plant = station.GetSubsystemByName("plant")
gripper = plant.GetBodyByName("hand_root")
plant_context = plant.GetMyContextFromRoot(context)

X_WG = plant.EvalBodyPoseInWorld(plant_context, gripper)
p_WG_pre = plant.EvalBodyPoseInWorld(plant_context, gripper).translation()
#p_WG_post = RigidTransform(RotationMatrix().MakeZRotation(90), [p_WG_pre[0], p_WG_pre[1], -0.02])
p_WG_post = [0.5, 0, 0.2]

allegro.set_positions(["thumb_revolute_y", "thumb_revolute_z", "thumb_1"], [1.2, 0.5, 0.5])
simulator.AdvanceTo(context.get_time() + 0.5)

poses = []
for i in range(15):
    translation = [p_WG_post[0] - 0.01 * i, p_WG_post[1], p_WG_post[2]]
    rotation = RotationMatrix().MakeXRotation(np.pi)
    poses.append(RigidTransform(rotation, translation))
    

move_arm(p_WG_post, simulator, station, context, poses=None, time_interval=0.5, frame="iiwa_link_6", arc_height=0.01)
move_arm(p_WG_post, simulator, station, context, poses=poses, time_interval=0.5, frame="hand_root", arc_height=0.01)
#allegro.set_positions(["thumb_revolute_z"], [.9])
#simulator.AdvanceTo(context.get_time() + .5)
#allegro.close_hand()
simulator.AdvanceTo(context.get_time() + .5)


RuntimeError: Inverse kinematics failed.

In [73]:
reset_arm(station, simulator, context, x0)
plant = station.GetSubsystemByName("plant")
gripper = plant.GetBodyByName("hand_root")
plant_context = plant.GetMyContextFromRoot(context)

X_WG = plant.EvalBodyPoseInWorld(plant_context, gripper)
p_WG_pre = plant.EvalBodyPoseInWorld(plant_context, gripper).translation()
#p_WG_post = RigidTransform(RotationMatrix().MakeZRotation(90), [p_WG_pre[0], p_WG_pre[1], -0.02])
p_WG_post = [0.5, 0, 0.2]

allegro.set_positions(["thumb_revolute_y", "thumb_revolute_z", "thumb_1"], [1.2, 0.5, 0.5])
simulator.AdvanceTo(context.get_time() + 0.5)

poses = []
for i in range(15):
    translation = [p_WG_post[0] - 0.01 * i, p_WG_post[1], p_WG_post[2]]
    rotation = RotationMatrix().MakeXRotation(np.pi)
    poses.append(RigidTransform(rotation, translation))
    
poses = list(reversed(poses))

move_arm(poses[0].translation(), simulator, station, context, poses=None, time_interval=0.5, frame="iiwa_link_6", arc_height=0.01)
move_arm(p_WG_post, simulator, station, context, poses=poses, time_interval=0.5, frame="hand_root", arc_height=0.01)
#allegro.set_positions(["thumb_revolute_z"], [.9])
#simulator.AdvanceTo(context.get_time() + .5)
#allegro.close_hand()
simulator.AdvanceTo(context.get_time() + .5)


RuntimeError: Inverse kinematics failed.

In [92]:
reset_arm(station, simulator, context, x0)
plant = station.GetSubsystemByName("plant")
gripper = plant.GetBodyByName("hand_root")
plant_context = plant.GetMyContextFromRoot(context)

X_WG = plant.EvalBodyPoseInWorld(plant_context, gripper)
p_WG_pre = plant.EvalBodyPoseInWorld(plant_context, gripper).translation()
#p_WG_post = RigidTransform(RotationMatrix().MakeZRotation(90), [p_WG_pre[0], p_WG_pre[1], -0.02])
p_WG_post = [0.4, 0, 0.2]

allegro.set_positions(["thumb_revolute_y", "thumb_revolute_z", "thumb_1"], [1.2, 0.5, 0.5])
simulator.AdvanceTo(context.get_time() + 0.5)

poses = []
for i in range(10):
    translation = [p_WG_post[0] - 0.01 * i, p_WG_post[1], p_WG_post[2]]
    rotation = RotationMatrix().MakeXRotation(np.pi)
    poses.append(RigidTransform(rotation, translation))
    
poses = list(reversed(poses))

move_arm(poses[0].translation(), simulator, station, context, poses=[poses[0]], time_interval=0.5, frame="iiwa_link_6", arc_height=0.01)
# move_arm(p_WG_post, simulator, station, context, poses=poses, time_interval=0.5, frame="hand_root", arc_height=0.01)
#allegro.set_positions(["thumb_revolute_z"], [.9])
#simulator.AdvanceTo(context.get_time() + .5)
#allegro.close_hand()

for i in range(10):
    allegro.moonwalk_index_lead()
    simulator.AdvanceTo(context.get_time() + .5)
    allegro.moonwalk_index_back()
    simulator.AdvanceTo(context.get_time() + .5)
    forward_joint = 3

    # # Move iiwa forward
    # current_state = station.GetOutputPort("iiwa+allegro.state_estimated").Eval(context)
    # current_state[forward_joint] += 0.05
    # current_state[5] += 0.05
    # station.GetInputPort("iiwa+allegro.desired_state").FixValue(context, current_state)
    # simulator.AdvanceTo(context.get_time() + .5)

    allegro.moonwalk_middle_lead()
    simulator.AdvanceTo(context.get_time() + .5)
    allegro.moonwalk_middle_back()
    simulator.AdvanceTo(context.get_time() + .5)

    # # Move iiwa forward
    # current_state = station.GetOutputPort("iiwa+allegro.state_estimated").Eval(context)
    # current_state[forward_joint] += 0.05
    # current_state[5] += 0.05
    # station.GetInputPort("iiwa+allegro.desired_state").FixValue(context, current_state)
    # simulator.AdvanceTo(context.get_time() + .5)
    
    allegro.moonwalk_pinky_lead()
    simulator.AdvanceTo(context.get_time() + .1)
    allegro.moonwalk_pinky_back()
    simulator.AdvanceTo(context.get_time() + .1)
    
    # Move iiwa forward
    current_state = station.GetOutputPort("iiwa+allegro.state_estimated").Eval(context)
    current_state[forward_joint] += 0.05
    current_state[5] += 0.05
    current_state[1] += 0.01
    station.GetInputPort("iiwa+allegro.desired_state").FixValue(context, current_state)
    simulator.AdvanceTo(context.get_time() + .5)


In [155]:
move_arm(p_WG_pre, simulator, station, context, time_interval=0.5, frame="iiwa_link_6", arc_height=0.01)

In [132]:

make_box_grip(p_W_brick, R_W_brick, allegro, station, context)
simulator.AdvanceTo(context.get_time() + 1)
move_arm(p_WG_pre, simulator, station, context, time_interval=0.5, frame="hand_root", arc_height=0.01)


[-0.25  0.    0.05] [0.75 0.   0.05] [0.75 0.   0.05] [ 0.75 -1.    0.05]


RuntimeError: Inverse kinematics failed.

In [60]:
X_WG.rotation()

RotationMatrix([
  [0.9996541431539786, 0.007020243955324362, 0.025343840472464574],
  [-0.00701367217911951, 0.9999753432800637, -0.0003481874250376038],
  [-0.025345659937153875, 0.00017031361319899232, 0.9996787326514575],
])

In [151]:
station.GetOutputPort("body_poses").Eval(context)[6]

RigidTransform(
  R=RotationMatrix([
    [-0.09021568530711165, -0.030771656225939803, 0.9954467516134058],
    [0.008957714387879847, -0.999507194059979, -0.030085351506888233],
    [0.9958819656352424, 0.006202757085396569, 0.09044687018922322],
  ]),
  p=[0.26503932022056576, -0.00888072687266526, 0.7128243501800458],
)

In [152]:
plant.GetBodyByName("iiwa_link_6").body_frame().GetFixedPoseInBodyFrame(plant_context)

TypeError: GetFixedPoseInBodyFrame(): incompatible function arguments. The following argument types are supported:
    1. (self: pydrake.multibody.tree.Frame) -> pydrake.math.RigidTransform
    2. (self: pydrake.multibody.tree.Frame) -> pydrake.math.RigidTransform

Invoked with: <BodyFrame name='iiwa_link_6' index=7 model_instance=2>, <pydrake.systems.framework.LeafContext object at 0x15600d230>

In [39]:

contact.get_contacts(station, context)

array([[ 6.23897108e-01,  6.52843052e-02, -3.54828327e-07],
       [ 6.62806992e-01,  6.99062311e-02, -1.60340233e-05],
       [ 5.09089070e-01,  4.95168755e-02, -1.24869511e-04],
       [ 4.60982275e-01,  1.18640410e-01, -9.92332021e-04],
       [ 4.60630810e-01,  1.20183633e-01, -9.81958293e-04],
       [ 6.68096023e-01,  1.81247187e-02, -1.29082907e-05],
       [ 6.27666443e-01, -5.89975599e-02, -5.60820181e-07],
       [ 6.66682288e-01, -6.12066078e-02, -1.23313829e-05]])